# 🔢 Tutorial: Encoding (Zamiana tekstu na liczby)

Algorytmy Machine Learning (Regresja, Sieci Neuronowe) są matematyczne. Nie rozumieją słów takich jak "Czerwony", "Mały" czy "Toyota". Rozumieją tylko liczby.

Proces zamiany tekstu (kategorii) na liczby to **Encoding**.
W tym notatniku nauczysz się 3 głównych metod i dowiesz się, kiedy ich używać, żeby nie popsuć modelu:

1.  **Ordinal Encoding:** Gdy kolejność MA znaczenie (Słaby < Średni < Dobry).
2.  **Label Encoding:** Szybka metoda, ale niebezpieczna dla cech bez kolejności.
3.  **One-Hot Encoding:** Najbezpieczniejsza dla danych nominalnych (Kolory, Miasta), ale "puchnie" pamięć.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder

# Ustawienia wyświetlania, żeby widzieć wszystkie kolumny
pd.set_option('display.max_columns', None)

print("Gotowe. Tworzymy dane...")

Gotowe. Tworzymy dane...


## KROK 1: Przygotowanie Danych (Laboratorium)

Stworzymy DataFrame, który zawiera każdy rodzaj danych kategorycznych:
1.  **Rozmiar (Ordinal):** Mały, Średni, Duży (Tu jest wyraźna hierarchia).
2.  **Kolor (Nominal):** Czerwony, Niebieski, Zielony (Tu NIE ma hierarchii - żaden kolor nie jest "lepszy").
3.  **Marka (Nominal):** Toyota, Ford, BMW.

In [2]:
df = pd.DataFrame({
    'ID_Klienta': [1, 2, 3, 4, 5],
    'Rozmiar': ['Mały', 'Duży', 'Średni', 'Mały', 'Duży'],  # Kolejność ma znaczenie!
    'Kolor': ['Czerwony', 'Niebieski', 'Zielony', 'Czerwony', 'Niebieski'], # Brak kolejności
    'Marka': ['Toyota', 'BMW', 'Ford', 'BMW', 'Toyota']    # Brak kolejności
})

print("--- ORYGINAŁ ---")
display(df)

--- ORYGINAŁ ---


,ID_Klienta,Rozmiar,Kolor,Marka
0,1,Mały,Czerwony,Toyota
1,2,Duży,Niebieski,BMW
2,3,Średni,Zielony,Ford
3,4,Mały,Czerwony,BMW
4,5,Duży,Niebieski,Toyota


## Metoda 1: Ordinal Encoding (Kodowanie Porządkowe)

Stosujemy, gdy wiemy, że **A < B < C**.
Jeśli użyjemy tu losowych liczb, model zgłupieje. Musimy mu powiedzieć wprost: Mały to 0, Średni to 1, Duży to 2.

Najlepszy sposób? **Ręczny mapping** (słownik). Daje 100% kontroli.

In [3]:
df_ordinal = df.copy()

# Definiujemy naszą hierarchię
kolejnosc_rozmiaru = {
    'Mały': 0,
    'Średni': 1,
    'Duży': 2
}

# Mapujemy
df_ordinal['Rozmiar_Encoded'] = df_ordinal['Rozmiar'].map(kolejnosc_rozmiaru)

print("--- ORDINAL ENCODING ---")
print("Mały=0, Średni=1, Duży=2 (Zachowaliśmy sens danych)")
display(df_ordinal[['Rozmiar', 'Rozmiar_Encoded']])

--- ORDINAL ENCODING ---
Mały=0, Średni=1, Duży=2 (Zachowaliśmy sens danych)


,Rozmiar,Rozmiar_Encoded
0,Mały,0
1,Duży,2
2,Średni,1
3,Mały,0
4,Duży,2


## Metoda 2: Label Encoding (Pułapka Alfabetyczna)

`LabelEncoder` zamienia każdy unikalny tekst na liczbę (0, 1, 2...).
Zazwyczaj robi to **alfabetycznie**.

**ZAGROŻENIE:**
Jeśli użyjesz tego dla koloru:
*   Czerwony -> 0
*   Niebieski -> 1
*   Zielony -> 2

Model matematyczny pomyśli: *"Aha! Zielony (2) jest dwa razy ważniejszy/większy niż Niebieski (1)!"*. **To błąd logiczny**, który może zepsuć predykcję (szczególnie w Regresji i Sieciach Neuronowych). Drzewa decyzyjne (Random Forest) radzą sobie z tym nieco lepiej, ale wciąż jest to ryzykowne.

*Zasada: LabelEncoder stosuj głównie do zmiennej celu (y), a nie do cech (X).*

In [4]:
df_label = df.copy()
le = LabelEncoder()

# Encodujemy 'Kolor'
df_label['Kolor_Encoded'] = le.fit_transform(df_label['Kolor'])

print("--- LABEL ENCODING ---")
print("Przypisało liczby alfabetycznie: Czerwony(0), Niebieski(1), Zielony(2).")
print("Model może pomyśleć, że Zielony > Czerwony. Uwaga na to!")
display(df_label[['Kolor', 'Kolor_Encoded']])

--- LABEL ENCODING ---
Przypisało liczby alfabetycznie: Czerwony(0), Niebieski(1), Zielony(2).
Model może pomyśleć, że Zielony > Czerwony. Uwaga na to!


,Kolor,Kolor_Encoded
0,Czerwony,0
1,Niebieski,1
2,Zielony,2
3,Czerwony,0
4,Niebieski,1


## Metoda 3: One-Hot Encoding (Bezpieczna Opcja)

Skoro nie możemy nadać kolorom liczb 0, 1, 2 (bo sugeruje to kolejność), to co zrobić?
Tworzymy **osobną kolumnę dla każdego koloru**.

*   Czy_Czerwony: 1 lub 0
*   Czy_Zielony: 1 lub 0
*   Czy_Niebieski: 1 lub 0

**Zaleta:** Nie wprowadzamy fałszywej hierarchii.
**Wada:** Jeśli masz kolumnę "Miasto" z 1000 miast, stworzysz 1000 nowych kolumn! (Klątwa Wymiarowości).

In [5]:
# Używamy pandas get_dummies (najprostszy sposób na One-Hot)
# prefix='Kolor' doda nazwę przed kategorią, żebyśmy się nie pogubili
df_onehot = pd.get_dummies(df, columns=['Kolor'], prefix='Kolor', dtype=int)

print("--- ONE-HOT ENCODING ---")
print("Zamiast jednej kolumny 'Kolor', mamy trzy kolumny binarne.")
display(df_onehot)

--- ONE-HOT ENCODING ---
Zamiast jednej kolumny 'Kolor', mamy trzy kolumny binarne.


,ID_Klienta,Rozmiar,Marka,Kolor_Czerwony,Kolor_Niebieski,Kolor_Zielony
0,1,Mały,Toyota,1,0,0
1,2,Duży,BMW,0,1,0
2,3,Średni,Ford,0,0,1
3,4,Mały,BMW,1,0,0
4,5,Duży,Toyota,0,1,0


## Profesjonalny Tip: Pułapka Zmiennych Fikcyjnych (Dummy Variable Trap)

Zauważ, że w powyższym przykładzie One-Hot Encoding stworzyliśmy **nadmiar informacji**.
Mamy kolory: Czerwony, Niebieski, Zielony.
Jeśli wiemy, że:
*   `Kolor_Czerwony` = 0
*   `Kolor_Niebieski` = 0

...to matematycznie wynika z tego wprost, że **musi to być Zielony!** Nie potrzebujemy kolumny `Kolor_Zielony`, żeby to wiedzieć.

W modelach liniowych (Regresja Liniowa, Logistyczna) ten nadmiar nazywa się **współliniowością** i powoduje błędy obliczeniowe. Dlatego stosujemy parametr `drop_first=True`, który usuwa pierwszą kolumnę. Ta usunięta kolumna staje się tzw. **bazą (punktem odniesienia)**.

In [6]:
# One-Hot Encoding z usunięciem pierwszej kolumny (drop_first=True)
df_drop = pd.get_dummies(df, columns=['Kolor'], prefix='Kolor', drop_first=True, dtype=int)

print("--- ONE-HOT ENCODING (drop_first=True) ---")
print("Zauważ, że zniknął 'Kolor_Czerwony'. Stał się on naszą bazą.")
print("Jeśli Niebieski=0 i Zielony=0, to wiersz oznacza Czerwony.")
display(df_drop)

--- ONE-HOT ENCODING (drop_first=True) ---
Zauważ, że zniknął 'Kolor_Czerwony'. Stał się on naszą bazą.
Jeśli Niebieski=0 i Zielony=0, to wiersz oznacza Czerwony.


,ID_Klienta,Rozmiar,Marka,Kolor_Niebieski,Kolor_Zielony
0,1,Mały,Toyota,0,0
1,2,Duży,BMW,1,0
2,3,Średni,Ford,0,1
3,4,Mały,BMW,0,0
4,5,Duży,Toyota,1,0


## 🧠 Podsumowanie: Co wybrać?

| Sytuacja | Metoda | Przykład |
| :--- | :--- | :--- |
| Dane mają rangę / kolejność | **Ordinal Encoding** | Wykształcenie (Podst., Śred., Wyż.), Ocena (1-5) |
| Dane to kategorie (Marka, Miasto, Kolor) | **One-Hot Encoding** | Kolor Oczu, Płeć, Województwo |
| Kodujemy Target (y) w klasyfikacji | **Label Encoding** | Czy Kupil (Tak/Nie), Gatunek Irisa |
| Masz 5000 kategorii (np. Kod Pocztowy) | **Target Encoding** (Advanced)* | One-Hot by "zabił" pamięć RAM |

*\*Target Encoding to technika zaawansowana, gdzie zamieniamy kategorię na średnią wartość celu dla tej kategorii.*

## 📝 Zadanie Domowe

Sprawdź, dlaczego One-Hot Encoding może być niebezpieczny dla pamięci RAM.

**Zadanie:**
1. Stwórz DataFrame z jedną kolumną `Kod_Pocztowy`.
2. Wygeneruj w niej 10,000 losowych kodów (użyj `np.random.randint(10000, 99999, 1000)`).
3. Użyj `pd.get_dummies` na tej kolumnie.
4. Sprawdź wymiary nowej tabeli (`df.shape`). Ile kolumn powstało?